In [3]:
import numpy as np
import pandas as pd
import nhl_stats as nhl
import matplotlib.pyplot as plt
import pprint

### Generate & Save Teams Dataframe

In [7]:
teams = nhl.get_teams()
team_names = []
team_ids  = []
teams_df = pd.DataFrame()
for team in teams:
    team_names.append(team['name'])
    team_ids.append(team['id'])

teams_df['Names'] = team_names
teams_df['Ids'] = team_ids

teams_df.to_csv('csv/teams.csv')

### Generate Edmonton Oilers Specific Data

In [22]:
OILERS_ID = 22

# parallell arrays
oilers_roster = nhl.get_team_roster(team_id=OILERS_ID)
oilers_player_info = [nhl.get_player_info(player_id=player['person']['id']) for player in oilers_roster]
oilers_player_stats = [nhl.get_player_stats_for_season(player_id=player['person']['id'], season=2021) for player in oilers_roster]



### Generate Player DataFrame for Edmonton Oilers

In [32]:
combined_oilers_player_data = []
for i in range(len(oilers_roster)):
    try:
        player_stats = oilers_player_stats[i][0]['splits'][0]['stat']
        player_info = oilers_player_info[i]
        
        # combine data, append to list
        combined = {**player_info, **player_stats}
        combined_oilers_player_data.append(combined)
    except Exception as e:
        continue

In [6]:
oilers_players_df = pd.DataFrame(combined_oilers_player_data)
oilers_players_df.to_csv("csv/oilers_players_info_and_stats.csv")

NameError: name 'combined_oilers_player_data' is not defined

### Generate Game DataFrame for Edmonton Oilers

In [36]:
OILERS_ID = 22
oilers_game_schedule = nhl.get_game_schedule_between_dates_for_team_id(OILERS_ID, "2019-07-01", "2021-11-01")

In [37]:
all_games = []
for game in oilers_game_schedule:
    game_info = nhl.get_game_info(game_id=game['games'][0]['gamePk'])
    all_games.append(game_info)

In [38]:
pprint.pprint(all_games[0]['liveData']['boxscore']['teams']['home']['teamStats']['teamSkaterStats'])

{'blocked': 9,
 'faceOffWinPercentage': '52.9',
 'giveaways': 11,
 'goals': 2,
 'hits': 34,
 'pim': 4,
 'powerPlayGoals': 0.0,
 'powerPlayOpportunities': 0.0,
 'powerPlayPercentage': '0.0',
 'shots': 33,
 'takeaways': 10}


#### Team Statistics Per Game

In [39]:
edmontonData = {}
againstData = {}
teamStats = []

for game in all_games:
    if not game['liveData']['boxscore']['teams']['home']['team']['triCode'] == 'EDM':
        edmontonData = game['liveData']['boxscore']['teams']['away']['teamStats']['teamSkaterStats']        
        againstData = game['liveData']['boxscore']['teams']['home']['teamStats']['teamSkaterStats'] 
        teamStats.append((edmontonData, againstData))
    else:
        edmontonData = game['liveData']['boxscore']['teams']['home']['teamStats']['teamSkaterStats'] 
        againstData = game['liveData']['boxscore']['teams']['away']['teamStats']['teamSkaterStats'] 
        teamStats.append((edmontonData, againstData))

In [40]:
pprint.pprint(teamStats[0][0])

{'blocked': 9,
 'faceOffWinPercentage': '52.9',
 'giveaways': 11,
 'goals': 2,
 'hits': 34,
 'pim': 4,
 'powerPlayGoals': 0.0,
 'powerPlayOpportunities': 0.0,
 'powerPlayPercentage': '0.0',
 'shots': 33,
 'takeaways': 10}


In [41]:
combinedStats = []

for statPair in teamStats:
    edmontonStats = statPair[0]
    againstStats  = statPair[1]
    
    edmtKeys  = list(edmontonStats.keys())
    agnstKeys = list(againstStats.keys())
    
    for key in edmtKeys:
        edmontonStats['edmonton_' + key] = edmontonStats[key]
        del edmontonStats[key]
        
    for key in agnstKeys:
        againstStats['against_' + key] = againstStats[key]
        del againstStats[key]
        
    combined = {**edmontonStats, **againstStats}
    combinedStats.append(combined)

In [42]:
pprint.pprint(combinedStats[0])

{'against_blocked': 17,
 'against_faceOffWinPercentage': '47.1',
 'against_giveaways': 7,
 'against_goals': 0,
 'against_hits': 34,
 'against_pim': 0,
 'against_powerPlayGoals': 0.0,
 'against_powerPlayOpportunities': 2.0,
 'against_powerPlayPercentage': '0.0',
 'against_shots': 25,
 'against_takeaways': 2,
 'edmonton_blocked': 9,
 'edmonton_faceOffWinPercentage': '52.9',
 'edmonton_giveaways': 11,
 'edmonton_goals': 2,
 'edmonton_hits': 34,
 'edmonton_pim': 4,
 'edmonton_powerPlayGoals': 0.0,
 'edmonton_powerPlayOpportunities': 0.0,
 'edmonton_powerPlayPercentage': '0.0',
 'edmonton_shots': 33,
 'edmonton_takeaways': 10}


In [43]:
oilersGameStatsDf = pd.DataFrame(combinedStats)

In [44]:
oilersGameStatsDf

,edmonton_goals,edmonton_pim,edmonton_shots,edmonton_powerPlayPercentage,edmonton_powerPlayGoals,edmonton_powerPlayOpportunities,edmonton_faceOffWinPercentage,edmonton_blocked,edmonton_takeaways,edmonton_giveaways,...,against_pim,against_shots,against_powerPlayPercentage,against_powerPlayGoals,against_powerPlayOpportunities,against_faceOffWinPercentage,against_blocked,against_takeaways,against_giveaways,against_hits
0,2,4,33,0.0,0.0,0.0,52.9,9,10,11,...,0,25,0.0,0.0,2.0,47.1,17,2,7,34
1,2,4,37,0.0,0.0,2.0,72.1,5,1,4,...,4,24,0.0,0.0,2.0,27.9,31,7,12,26
2,1,12,27,0.0,0.0,4.0,43.9,14,9,17,...,8,34,33.3,2.0,6.0,56.1,10,5,10,21
3,6,8,29,100.0,1.0,1.0,43.1,11,9,14,...,2,20,0.0,0.0,4.0,56.9,13,9,14,31
4,2,10,40,0.0,0.0,2.0,70.2,17,7,13,...,4,33,0.0,0.0,5.0,29.8,11,10,16,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,5,18,31,100.0,2.0,2.0,56.4,10,11,6,...,4,28,0.0,0.0,4.0,43.6,14,8,10,25
155,5,6,33,33.3,1.0,3.0,53.7,16,10,9,...,6,39,0.0,0.0,3.0,46.3,14,10,12,22
156,3,4,37,25.0,1.0,4.0,42.3,7,12,19,...,8,34,0.0,0.0,2.0,57.7,26,6,14,27
157,2,2,34,100.0,2.0,2.0,50.0,15,1,5,...,4,30,0.0,0.0,1.0,50.0,13,7,9,25


In [45]:
oilersGameStatsDf.to_csv("csv/oilersGameStatistics.csv")